> Martin Fridrich, 03/2022 

# Final project


### Data loading and initial transformations

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from itertools import product
import re
from fuzzywuzzy import fuzz

In [ ]:
DATA_DIR = "../data/final_project/"
# sales
sales_train = pd.read_csv(DATA_DIR+"sales_train.csv")
sales_test = pd.read_csv(DATA_DIR+"test.csv")
# addional info
items = pd.read_csv(DATA_DIR+"items.csv")
item_categories = pd.read_csv(DATA_DIR+"item_categories.csv")
shops = pd.read_csv(DATA_DIR+"shops.csv")

In [ ]:
# downcast dataframe
def optimize_numeric_dtypes(df):
    import pandas as pd
    float_cols = df.select_dtypes("float").columns
    int_cols = df.select_dtypes("integer").columns
    df[float_cols] = df[float_cols].\
        apply(pd.to_numeric, downcast="float")
    df[int_cols] = df[int_cols].\
        apply(pd.to_numeric, downcast="integer")
    return df

def optimize_categories(df):
    import pandas as pd
    cat_cols = df.select_dtypes("object").columns
    df[cat_cols] = df[cat_cols].\
        apply(lambda x: x.astype("category").cat.codes)
    return df    

In [ ]:
# duplicate shops
shop_duplicates_map = {0: 57, 1: 58, 11: 10, 40: 39}
sales_train["shop_id"] = sales_train["shop_id"].replace(shop_duplicates_map)
# datetime
sales_train["date"] = pd.to_datetime(sales_train["date"], format="%d.%m.%Y")
# filtering on test set
sales_train = sales_train.loc[sales_train.item_id.isin(sales_test["item_id"].unique()), :]
# drop training items with extreme or negative prices or sales counts
sales_train = sales_train[((sales_train["item_price"] > 0) & (sales_train["item_price"] < 50000)) &
    ((sales_train["item_cnt_day"] > 0) & (sales_train["item_cnt_day"] < 1000))]
# test-based cartesian product & inds
sales_index = pd.DataFrame(product(sales_test.item_id.unique(), sales_test.shop_id.unique(),
    range(35)),columns = ["item_id", "shop_id", "date_block_num"])
sales_index = optimize_numeric_dtypes(sales_index)
# sales table
sales_train["revenue"] = sales_train["item_price"]*sales_train["item_cnt_day"]
sales = sales_train.groupby(["shop_id", "item_id", "date_block_num"], as_index=False).agg(
    total_sold=("item_cnt_day", sum),  sum_revenue=("revenue",sum),
    n_transactions = ("item_cnt_day",pd.Series.count))\
        .sort_values("date_block_num")
sales = sales_index.merge(sales, how="left",
    on=["shop_id", "item_id", "date_block_num"])
sales["date_block_num"].fillna(34, inplace=True)
sales.loc[sales.date_block_num<34, ["total_sold", "sum_revenue", "n_transactions"]]=\
    sales.loc[sales.date_block_num<34, ["total_sold", "sum_revenue", "n_transactions"]].fillna(0)
sales = optimize_numeric_dtypes(sales)
del sales_train, sales_test, sales_index; gc.collect()
sales.head(3) 

# Exploration

# Feature engineering

In [ ]:
# item groups based on adjacency and similarity
items.item_name = items.item_name.apply(lambda x: re.sub("[^A-ZА-Яa-zа-я0-9 ]","", x))
items.item_name = items.item_name.apply(lambda x: re.sub("\\s+"," ", x))
item_names = pd.concat([items.item_id, items.item_name,
    items.item_id.shift().fillna(0), items.item_name.shift().fillna("")], axis=1)
item_names.columns = ["item_id", "item_name", "previous_item_id", "previous_item_name"]
item_names["similarity"] = item_names.apply(\
     lambda x: fuzz.ratio(x["item_name"],x["previous_item_name"]), axis=1)
item_names["group_start"] = item_names["similarity"] <= 75
item_group_map = item_names.loc[item_names.group_start, ["item_id"]]
item_group_map.columns = ["group_start"]
item_group_map["group_end"] = item_group_map["group_start"].shift(-1).\
    fillna(item_group_map["group_start"].max()+1).astype("int")
item_group_map["item_sim_id"] = list(range(len(item_group_map)))
# remap
items = items.merge(item_group_map, how="left",
    left_on=["item_id"], right_on=["group_start"])
items = items.sort_values("item_id")
items["item_sim_id"] = items["item_sim_id"].fillna(method="ffill").astype("int")
del item_names, item_group_map; gc.collect()
items = items[["item_id","item_name", "item_category_id", "item_sim_id"]]
# name length
items["item_name_len"] = items.item_name.apply(len).astype("int16")
items.head(3)

In [ ]:
# categories
cat_splits = item_categories.item_category_name.apply(lambda x: x.split(" - "))
item_categories = pd.concat([item_categories,
    pd.DataFrame([c if len(c)>1 else [c[0],""] for c in cat_splits],
        columns=["parent_cat","child_cat"])], axis=1)
items = items.merge(item_categories, how="inner")
# return categorical encoding
items = optimize_categories(items[["item_id", "item_category_id", "item_sim_id",
    "item_name_len", "parent_cat", "child_cat"]])
# add categories
sales = sales.merge(items, how="inner", on="item_id")
del item_categories, items; gc.collect()
sales.head(3)

In [ ]:
# date features
def working_days_count(from_date, to_date=None):
    import pandas as pd
    if to_date is None:
        to_date=from_date+pd.DateOffset(months=1)
    temp_date=from_date
    cnt=0
    while temp_date<to_date:
        if temp_date.weekday()<5:
            cnt+=1
        temp_date+=pd.DateOffset(days=1)
    return ((to_date-from_date).days, cnt, (to_date-from_date).days-cnt)
dates = pd.DataFrame(sales.date_block_num.unique(), columns=["date_block_num"])
dates["date"] = dates["date_block_num"].apply(lambda x: pd.to_datetime("01/01/2013")+\
    pd.DateOffset(months=x))
dates["year"] = dates.date.dt.year
dates["month"] = dates.date.dt.month
dates = optimize_numeric_dtypes(pd.concat([dates, pd.DataFrame.from_records(
        dates["date"].apply(lambda x: working_days_count(x)),
    columns=["total_days", "working_days", "weekend_days"])], axis=1))
sales = sales.merge(dates[["date_block_num","year","month","total_days", "working_days",
    "weekend_days"]], how="inner", on="date_block_num")
del dates; gc.collect()
sales.tail(3)    

In [ ]:
# pricing overall and inside cat
item_pricing = sales.groupby(["date_block_num","item_id","item_category_id"], as_index=False).\
    agg(total_sold=("total_sold", sum),  sum_revenue=("sum_revenue",sum))
train_ind = item_pricing.date_block_num<34
item_pricing.loc[train_ind,"average_price"] = item_pricing["sum_revenue"]/item_pricing["total_sold"]
item_pricing.loc[train_ind,"average_price"]  = item_pricing.groupby(["item_id"])["average_price"].\
    apply(lambda x: x.fillna(method="ffill").fillna(method="bfill"))
# add price changes
item_pricing.loc[train_ind,"price_change"] = item_pricing.groupby(["item_id"])["average_price"].\
    apply(lambda x: x/x.shift()).fillna(-1)
# add position within the category
item_pricing.loc[train_ind,"price_perc"] = item_pricing.groupby(["item_category_id", "date_block_num"])\
    ["average_price"].apply(lambda x: x.rank(pct=True)).fillna(-1)
item_pricing = optimize_numeric_dtypes(item_pricing[["date_block_num", "item_id", "price_change",
    "price_perc"]])
sales = sales.merge(item_pricing, how="left",
    on=["date_block_num", "item_id"])
del item_pricing; gc.collect()
sales.tail(3)

In [ ]:
# shop location
shops["city_id"] = shops.shop_name.apply(lambda x:\
    re.sub("[^A-ZА-Яa-zа-я0-9 ]","", x.split(" ")[0]))
#shops["city_code_id"] = shops.shop_name.apply(lambda x:\
#    re.sub("[^A-ZА-Яa-zа-я0-9 ]","", x.split(" ")[1]))
# keep wrong codes, so city and code are not the same
# shops["city_code"][shops.city=="Якутск"] = "ТЦ"
shops = optimize_categories(shops[["shop_id", "city_id"]])
sales = sales.merge(shops, how="inner", on="shop_id")
del shops; gc.collect()
sales.tail(3)

In [ ]:
# time-series
# smoothed demand across various cats
train_ind = sales.date_block_num<34
# city
sales.loc[train_ind, "city_demand"] = sales[train_ind].\
    groupby(["city_id","date_block_num"])["total_sold"].apply(lambda x:
        x.ewm(span=3, min_periods=3).mean()).fillna(0)#.astype("float32")
# shop
sales.loc[train_ind, "shop_demand"] = sales[train_ind].\
    groupby(["shop_id","date_block_num"])["total_sold"].apply(lambda x:
        x.ewm(span=3, min_periods=3).mean()).fillna(0)#.astype("float32")
# categories
sales.loc[train_ind, "cat_demand0"] = sales[train_ind].\
    groupby(["parent_cat","date_block_num"])["total_sold"].apply(lambda x:
        x.ewm(span=3, min_periods=3).mean()).fillna(0)#.astype("float32")
# categories
sales.loc[train_ind, "cat_demand1"] = sales[train_ind].\
    groupby(["child_cat","date_block_num"])["total_sold"].apply(lambda x:
        x.ewm(span=3, min_periods=3).mean()).fillna(0)#.astype("float32")      
# categories
sales.loc[train_ind, "cat_demand2"] = sales[train_ind].\
    groupby(["item_sim_id","date_block_num"])["total_sold"].apply(lambda x:
        x.ewm(span=3, min_periods=3).mean()).fillna(0)#.astype("float32")        
# item
sales.loc[train_ind, "item_demand"] = sales[train_ind].\
    groupby(["item_id","date_block_num"])["total_sold"].apply(lambda x:
        x.ewm(span=3, min_periods=3).mean()).fillna(0)#.astype("float32")
# item maturity
sales["item_maturity"] = (sales["date_block_num"] - sales[["item_id","date_block_num"]].merge(\
    sales[sales.n_transactions>0].groupby("item_id", as_index=False).\
        agg(date_block_num_min=("date_block_num",min)),
            how="left")["date_block_num_min"]).clip(-1,34).fillna(-1)
del train_ind; gc.collect();
sales = optimize_numeric_dtypes(sales)
sales.tail(3)

In [ ]:
lag_steps = [1,2,3,6]
lag_fill = {"total_sold":0, "sum_revenue":0, "n_transactions":0, "price_change":1, "price_perc":-1,
    "city_demand":0, "shop_demand":0, "cat_demand0":0,"cat_demand1":0,"cat_demand2":0, "item_demand":0}
for l in lag_steps:
    for c in lag_fill.keys():
        sales.loc[:,"lag"+str(l)+"_"+c] = sales.groupby([sales.shop_id,sales.item_id])\
            [c].shift(l).fillna(lag_fill[c])
sales = sales[sales.date_block_num>25] # just reduce the dataset
target = "total_sold"
col_fil = [c for c in sales.columns if (c not in lag_fill.keys()) or (c ==target)]
sales = optimize_numeric_dtypes(sales.loc[:,col_fil])
sales.head(3)

In [ ]:
# some artificial feature interactions
from sklearn.preprocessing import PolynomialFeatures
from lightgbm import LGBMRegressor
lgbm_feature_imp = LGBMRegressor()
features = [c for c in sales.columns if (c not in [target])]
lgbm_feature_imp.fit(sales.loc[:,features], sales.loc[:,target])
feature_imp = pd.DataFrame.from_records(zip(lgbm_feature_imp.feature_name_,
    lgbm_feature_imp.feature_importances_),
    columns=["feature","imp"]).sort_values("imp", ascending=False)    
# use interactions between 5 more prominent features, as they will occur frequently
# on trees' decision path
pf = PolynomialFeatures(interaction_only=True, include_bias=False).\
    fit(sales.loc[:,feature_imp.feature[:5]])
feature_int = pd.DataFrame(pf.transform(sales.loc[:,feature_imp.feature[:5]]),
    columns=pf.get_feature_names_out(), index=sales.index)
feature_int = optimize_numeric_dtypes(feature_int.loc[:,[c for c in feature_int if " " in c]])
feature_int.columns = [c.replace(" ", "_") for c in feature_int.columns]
sales = pd.concat([sales, feature_int], axis=1)
del feature_int, lgbm_feature_imp, feature_imp, pf; gc.collect()

# Modeling pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, PolynomialFeatures
from category_encoders import TargetEncoder,PolynomialEncoder,CatBoostEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import IncrementalPCA
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression,ElasticNetCV,ElasticNet,BayesianRidge
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

In [ ]:
# stage filters
b_ind = (sales.year==2015) & (sales.month==10) # 2nd stage
c_ind = (sales.year==2015) & (sales.month==9)
test_ind = (sales.year==2015) & (sales.month==11) # prediction
a_ind = ~b_ind & ~c_ind & ~test_ind # 1st stage

# col filters for mean-encoding
target = "total_sold"
features = [c for c in sales.columns if (c not in [target])]
cat_cols = ["shop_id", "item_id", "date_block_num", "item_name_len",
    "item_category_id", "item_sim_id", "parent_cat", "child_cat",
    "year", "month", "total_days", "working_days", "weekend_days",
    "city_id"]
num_cols = [f for f in sales[features].columns if (f not in cat_cols) & (f not in [target])]
# clippin
sales[target] = sales[target].clip(0,20)

## First stage models

In [ ]:
# catboost on raw data
cat_model = CatBoostRegressor(cat_features=cat_cols,
    verbose=100)
cat_model.fit(sales.loc[a_ind,features], sales.loc[a_ind,target])

# pushout preds
cat_b_feature = cat_model.predict(sales.loc[b_ind,features]).clip(0,20)
cat_c_feature = cat_model.predict(sales.loc[c_ind,features]).clip(0,20)
cat_test_feature = cat_model.predict(sales.loc[test_ind,features]).clip(0,20)

In [ ]:
# lightgbm raw
lgbm_model = lgbm_model = Pipeline(steps=[("ct",ColumnTransformer(transformers=[
        ("se", StandardScaler(), num_cols),
        ("te", TargetEncoder(cols=cat_cols, drop_invariant=True,
            min_samples_leaf=5*10**4, smoothing=10**3), cat_cols)])),
    ("lgbm", LGBMRegressor(n_estimators=500, learning_rate=.01,
        max_depth=4, subsample=.5, num_leaves = 1024))])
lgbm_model.fit(sales.loc[a_ind,features], sales.loc[a_ind,target])

# pushout preds
lgbm_b_feature = lgbm_model.predict(sales.loc[b_ind,features]).clip(0,20)
lgbm_c_feature = lgbm_model.predict(sales.loc[c_ind,features]).clip(0,20)
lgbm_test_feature = lgbm_model.predict(sales.loc[test_ind,features]).clip(0,20)

In [ ]:
# lightgbm pca
lgbm_pca_model = lgbm_model = Pipeline(steps=[("ct",ColumnTransformer(transformers=[
        ("se", StandardScaler(), num_cols),
        ("te", TargetEncoder(cols=cat_cols, drop_invariant=True,
            min_samples_leaf=50000, smoothing=1000), cat_cols)])),
    ("qe", QuantileTransformer()),
    ("pca", IncrementalPCA(n_components=10)),
    ("lgbm", LGBMRegressor())])
lgbm_pca_model.fit(sales.loc[a_ind,features], sales.loc[a_ind,target])

# pushout preds
lgbm_pca_b_feature = lgbm_pca_model.predict(sales.loc[b_ind,features]).clip(0,20)
lgbm_pca_c_feature = lgbm_pca_model.predict(sales.loc[c_ind,features]).clip(0,20)
lgbm_pca_test_feature = lgbm_pca_model.predict(sales.loc[test_ind,features]).clip(0,20)

## Second stage model

In [ ]:
b_features = np.stack([cat_b_feature, lgbm_b_feature, lgbm_pca_b_feature], axis=1)
c_features = np.stack([cat_c_feature, lgbm_c_feature, lgbm_pca_c_feature], axis=1)
test_features = np.stack([cat_test_feature, lgbm_test_feature, lgbm_pca_test_feature], axis=1)

lr_model = LinearRegression()
lr_model.fit(b_features, sales.loc[b_ind,target])

train_rsme = np.mean((lr_model.predict(b_features).clip(0,20)-sales.loc[b_ind,target])**2)**(1/2)
val_rsme = np.mean((lr_model.predict(c_features).clip(0,20)-sales.loc[c_ind,target])**2)**(1/2)
print("Stacked pipeline> RMSE on train set: {}; on val set: {}".format(train_rsme, val_rsme))

# Submission

In [ ]:
test_mapping = pd.read_csv(DATA_DIR+"test.csv")
test_predictions = sales.loc[test_ind,["item_id", "shop_id"]]
test_predictions["item_cnt_month"] = lr_model.predict(test_features).clip(0,20)
test_predictions = test_mapping.merge(test_predictions, on=["item_id", "shop_id"],how="left").fillna(0)
test_predictions[["ID", "item_cnt_month"]].to_csv("../data/submissions/submission.csv", index=False)

In [ ]:
!kaggle competitions submit -c competitive-data-science-predict-future-sales -f ../data/submissions/submission.csv -m "Meh, but..."